In [5]:
import requests
import bs4
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import re
import pandas as pd
import numpy as np

In [8]:
#getting urls for each page
urls = []

#get the urls for each page
for page in tqdm_notebook (range(0,226,25)):
    url_page = 'https://transcripts.foreverdreaming.org/viewforum.php?f=845&start={}'.format(page)
    
    #get the url for each episode 
    add = 'https://transcripts.foreverdreaming.org/'
    r = requests.get(url_page)
    new = BeautifulSoup(r.text, 'html.parser')
    lines = []
    for link in new.find_all('td', class_='topic-titles row2'):
        url_new = link.find('a',class_='topictitle')['href']
        pattern = re.compile(r'(.(viewtopic.php\?f=845&t=\d\d\d\d\d))')
        url_short = re.findall(pattern, url_new)[0][1]
        urls.append(add + url_short)

In [3]:
#loop through the links for each episode
words = []
title = []
for url in tqdm_notebook(urls):
    r = requests.get(url)
    new = BeautifulSoup(r.text, 'html.parser')
    
    #go to each page and pull out the lines
    for item in tqdm_notebook(new.find_all('p')):
        try:
            words.append(item.text)
        except:
            words.append(np.nan)
        #find the season#, episode# and title of each episode
        for item in new.find_all('div', {'class':'boxheading'}):
            try:
                title.append(item.find('h2').text)
            except:
                title.append(np.nan)

In [4]:
#drop missing values creating the title column, assign to variable
lines = pd.DataFrame(title).dropna()
lines['lines'] = words

In [5]:
lines.to_csv('1_lines.csv', index = False)